In [11]:
import torch
import torch.nn as nn
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--context', type=int, default=100)
parser.add_argument('--future', type=int, default=10)

args = parser.parse_args()
print(args.context)
print(args.future)
def LinearCell(num_input, num_hidden, Dropout=0):
    Seq = nn.Sequential(
        nn.Linear(num_input,num_hidden),
        nn.LeakyReLU(0.8),
        nn.Dropout(Dropout)
    )
    return Seq

class Net(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layers, num_linear =1):
        """
        input_size: input size
        hidden_size: hidden size
        output_size: output size
        num_layers: number of layers
        num_linear: number of linear layers
        """
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers)
        LinearSeq = []
        for i in range(num_linear):
            LinearSeq.append(LinearCell(hidden_size,hidden_size,Dropout=0))
        self.LinearSeq = nn.Sequential(*LinearSeq)
        self.L1 = LinearCell(hidden_size,hidden_size,Dropout=0)
        self.L2 = LinearCell(hidden_size,hidden_size,Dropout=0)
        self.LOut = LinearCell(hidden_size,output_size)
        self.init_weights()
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.normal_(m.bias)
    def forward(self,x,future=0):
        """
        x: input [sequence_length,batch_size,input_size]
        sequence_length: sequence length
        batch_size: batch size
        input_size: input size == 15
        future: number of future predictions
        """
        # outputs
        outputs = []
        h_t = torch.zeros(self.num_layers,1,self.hidden_size)
        c_t = torch.zeros(self.num_layers,1,self.hidden_size)
        
        for input_t in x.split(1,dim=0):
            out, (h_t,c_t) = self.lstm1(input_t,(h_t,c_t))
            #print (out.shape)
            out = self.LinearSeq(out)
            l1 = self.L1(out)
            l2 = self.L2(l1)
            output = self.LOut(l2)
            outputs.append(output)
        
        for i in range(future):
            out, (h_t,c_t) = self.lstm1(output, (h_t,c_t))
            out = self.LinearSeq(out)
            l1 = self.L1 (out)
            l2 = self.L2(l1)
            output = self.LOut(l2)
            outputs.append(output)
        outputs = torch.cat(outputs,dim=0)
        return outputs
# load model weights
input_size = 15
argument = {
    'input_size' : input_size,
    'hidden_size' : 18,
    'output_size' : input_size,
    'num_layers' : 3,
    'num_linear' : 2
}
model = Net(**argument)
model.load_state_dict(torch.load('Model/modelApodaca4.2.pt'))



<All keys matched successfully>